<a href="https://colab.research.google.com/github/uhsipra/AntiVaCCers/blob/main/Collected_data_model_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# utilities
import re
import numpy as np
import pandas as pd
# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# nltk
from nltk.stem import WordNetLemmatizer
# sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

In [6]:
# Loading in collected data for each city
dataset_calgary = pd.read_pickle('Calgary.pkl')
dataset_edmonton = pd.read_pickle('Edmonton.pkl')
dataset_montreal = pd.read_pickle('Montreal.pkl')
dataset_ottawa = ppd.read_pickle('Ottoawa.pkl')
dataset_edmonton.sample(5)

,text
56,RT @ABBedClosures: Multiple DMs over the past ...
24,RT @MerlinofCanada: There are now 822 COVID-19...
51,RT @MerlinofCanada: There are now 822 COVID-19...
63,RT @MerlinofCanada: There are now 822 COVID-19...
34,@CyrilF24 @JBG_1979 I trust bitchute more than...


In [7]:
# We need to clean up the data by removing stopwords, emojis and converted to lowercase

# We will focus on only the text of the tweets and the sentiment (target)
dataset = dataset_edmonton


# Turning all tweets into lowercase
dataset['text'] = dataset['text'].str.lower()
dataset['text'].tail()
len(dataset['text'])

100

In [8]:
# Creating a stopwords set
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [9]:
# Removing the stop words from tweet text

STOPWORDS = set(stopwordlist)

def removing_stopwords(text):
  return " ".join([word for word in str(text).split() if word not in STOPWORDS])

dataset['text'] = dataset['text'].apply(lambda text: removing_stopwords(text))
dataset['text'].tail()

95    rt @selek437: go. omicron spread even faster k...
96    rt @merlinofcanada: 822 covid-19 patients hosp...
97    #alberta plans end single-site order continuin...
98    @don_coi @edmonton_wild @alexmunter @lindysmit...
99    rt @abbedclosures: multiple dms over past 24 h...
Name: text, dtype: object

In [10]:
# Removing puncuation

import string

english_punctuation = string.punctuation
punctuations_list = english_punctuation

def removing_punctuation(text):
  translator = str.maketrans('', '', punctuations_list)
  return text.translate(translator)

dataset['text'] = dataset['text'].apply(lambda text: removing_punctuation(text))
dataset['text'].tail()

95    rt selek437 go omicron spread even faster kill...
96    rt merlinofcanada 822 covid19 patients hospita...
97    alberta plans end singlesite order continuingc...
98    doncoi edmontonwild alexmunter lindysmithmd ch...
99    rt abbedclosures multiple dms over past 24 hou...
Name: text, dtype: object

In [11]:
# Removing URLs

def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
dataset['text'] = dataset['text'].apply(lambda x: cleaning_URLs(x))
dataset['text'].tail()

95    rt selek437 go omicron spread even faster kill...
96    rt merlinofcanada 822 covid19 patients hospita...
97    alberta plans end singlesite order continuingc...
98    doncoi edmontonwild alexmunter lindysmithmd ch...
99    rt abbedclosures multiple dms over past 24 hou...
Name: text, dtype: object

In [12]:
# Removing numbers

def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
dataset['text'] = dataset['text'].apply(lambda x: cleaning_numbers(x))
dataset['text'].tail()

95    rt selek go omicron spread even faster kill ol...
96    rt merlinofcanada  covid patients hospital inc...
97    alberta plans end singlesite order continuingc...
98    doncoi edmontonwild alexmunter lindysmithmd ch...
99    rt abbedclosures multiple dms over past  hours...
Name: text, dtype: object

In [13]:
# Tokensizing the text from tweets

dataset['text'] = dataset['text'].apply(lambda x: x.split())
dataset['text'].tail()

95    [rt, selek, go, omicron, spread, even, faster,...
96    [rt, merlinofcanada, covid, patients, hospital...
97    [alberta, plans, end, singlesite, order, conti...
98    [doncoi, edmontonwild, alexmunter, lindysmithm...
99    [rt, abbedclosures, multiple, dms, over, past,...
Name: text, dtype: object

In [14]:
# Stemming the data
import nltk
st = nltk.PorterStemmer()

def stem_data(data):
  text = [st.stem(word) for word in data]
  return data

dataset['text'] = dataset['text'].apply(lambda x: stem_data(x))
dataset['text'].tail()

95    [rt, selek, go, omicron, spread, even, faster,...
96    [rt, merlinofcanada, covid, patients, hospital...
97    [alberta, plans, end, singlesite, order, conti...
98    [doncoi, edmontonwild, alexmunter, lindysmithm...
99    [rt, abbedclosures, multiple, dms, over, past,...
Name: text, dtype: object

In [15]:
# Lemmatizing the data
nltk.download('wordnet')
lm = nltk.WordNetLemmatizer()

def lemm_data(data):
  text = [lm.lemmatize(word) for word in data]
  return data

dataset['text'] = dataset['text'].apply(lambda x: lemm_data(x))
dataset['text'].tail()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


95    [rt, selek, go, omicron, spread, even, faster,...
96    [rt, merlinofcanada, covid, patients, hospital...
97    [alberta, plans, end, singlesite, order, conti...
98    [doncoi, edmontonwild, alexmunter, lindysmithm...
99    [rt, abbedclosures, multiple, dms, over, past,...
Name: text, dtype: object

In [16]:
# TEST CODE
test = list(dataset['text'])
len(test)

100

In [17]:
# Concatonating the individual words into a single sentence


for i in range(len(test)):
  test[i] = " ".join(test[i])

dataset['cleantext'] = test
dataset['cleantext'].tail()

95    rt selek go omicron spread even faster kill ol...
96    rt merlinofcanada covid patients hospital incr...
97    alberta plans end singlesite order continuingc...
98    doncoi edmontonwild alexmunter lindysmithmd ch...
99    rt abbedclosures multiple dms over past hours ...
Name: cleantext, dtype: object

In [20]:
# Feature extraction

from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.9, min_df=2, max_features=500000, stop_words='english')
bow = bow_vectorizer.fit_transform(dataset['cleantext'])

Testing the city datasets on the trained models

In [19]:
# Loading in trained logistic regression model
import pickle

model_logreg = pickle.load(open('trained_logistic_regression_model.sav', 'rb'))

from sklearn.linear_model import LogisticRegression



Due to time constraints, could not complete the code.